 # KSP Heuristic Algorithms Comparison

In [260]:
%matplotlib inline

In [254]:
import numpy as np
import pickle
import random as rn
import copy
import time
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import namedtuple

In [216]:
"""
G = Maximum weight
N = Number of items
g = weights
v = values
"""
DS=namedtuple('DS','G,N,g,v')

In [217]:
def open_dataset(file):
    pickle_in = open(file, "rb")
    dataset = pickle.load(pickle_in)
    pickle_in.close()
    return dataset


 ## Backtracking method

 ### Algorithm

In [218]:
def knapsack(max_weight, weights, values, number_of_items):
    # Base Case
    if number_of_items == 0 or max_weight == 0:
        return 0

    # If weight of the nth item is more than Knapsack of capacity
    # W, then this item cannot be included in the optimal solution
    if (weights[number_of_items - 1] > max_weight):
        return knapsack(max_weight, weights, values, number_of_items - 1)
    else:
    # return the maximum of two cases:
    # (1) nth item included
    # (2) not included
        return max(values[number_of_items - 1] + knapsack(max_weight - weights[number_of_items - 1], weights, values, number_of_items - 1),
                   knapsack(max_weight, weights, values, number_of_items - 1))


 ### Plots & Measurements

 #### Measure Tb8

In [219]:
dataset = open_dataset("dataset_ds8.pickle")


In [220]:
tb8=time.process_time_ns()
knapsack(dataset.G , dataset.g , dataset.v , dataset.N)
tb8=time.process_time_ns() - tb8


In [221]:
dataset = open_dataset("dataset_ds10.pickle")

In [222]:
tb10=time.process_time_ns()
knapsack(dataset.G , dataset.g , dataset.v , dataset.N)
tb10 = time.process_time_ns() - tb10

In [224]:
print("Cpu Time \n\tTb8:\t{0}\t[ns] \n\tTb10:\t{1}\t[ns]".format(tb8,tb10))
print(tb10/tb8)

Cpu Time 
	Tb8:	1013538	[ns] 
	Tb10:	3516280	[ns]
3.469312448077921


10/8 = 1.25

Considering that the second dataset, DS10, is 1.25 bigger than the first, DS8, and the fraction TB10/TB8 = 2.46 we can observe that backtracking is not feasible on bigger datasets. It took 2.46 times more CPU time to find the optimum solution for DS10 than for DS8.

**Conclusion**: Considering that an increase in seach space by 1.25 times ( added 2 more elements) led to a increase by 2.46 times CPU time it can be concluded that backtracking algorithms are not feasible for datasets consisting of tens, hundreds, thousands of elements.

 ## Neighborhood Search Algorithm

 ### Algorithm

In [86]:
def generate_solution():
    return [np.random.randint(2) for x in range(0,dataset.N)]

In [87]:
def cost_function(solution):
    total_value = 0
    total_weight = 0
    for i in range(0,len(solution)):
        if solution[i] == 1:
            total_value += dataset.v[i]
            total_weight += dataset.g[i]
    if total_weight > dataset.G:
        return 0
    else:
        return total_value


In [88]:
def ns_compute_neighborhood(S):
    '''
    Since the neighborhood of a solution S is made of all  the solutions 
    that that can be obtaine by using only one simple transformation,
    I chose that the simple transformation to be a bit flip
    in current solution S. This means that a value of 1 in solution array represent
    an object that is in the knapsack while a value of 0 represents a missing object.
    '''
    solutions=list()
    for i in range(0,len(S)):
        new_solution=list(S)
        if S[i] == 0:
            new_solution[i] = 1
        else:
            new_solution[i] = 0
        value = cost_function(new_solution)
        solutions.append((new_solution,value))
    best_solution=max(solutions,key=lambda item:item[1])
    return best_solution[0],best_solution[1]


In [140]:
def ns():
    S = generate_solution()
    while True:
        new_solution,new_cost=ns_compute_neighborhood(S)
        old_cost = cost_function(S)
        if new_cost > old_cost:
            S = new_solution
        else:
            break
    return (S,cost_function(S))


 ### Plots & Measurements

In [137]:
dataset = open_dataset("dataset_ds8.pickle")
tns8=time.process_time_ns()
S=ns()
tns8=time.process_time_ns() - tns8
print("Solution {0} found in {1}[ns]".format(S[1],tns8))

Solution 288 found in 860060[ns]


In [135]:
dataset = open_dataset("dataset_ds10.pickle")
tns10=time.process_time_ns()
S=ns()
tns10=time.process_time_ns() - tns10
print("Solution {0} found in {1}[ns]".format(S[1],tns10))

Solution 305 found in 887888[ns]


In [141]:
dataset = open_dataset("dataset_ds50.pickle")
tns50=time.process_time_ns()
S=ns()
tns50=time.process_time_ns() - tns50
print("Solution {0} found in {1}[ns]".format(S[1],tns50))

Solution 0 found in 2004049[ns]


In [142]:
dataset = open_dataset("dataset_ds100.pickle")
tns100=time.process_time_ns()
S=ns()
tns100=time.process_time_ns() - tns100
print("Solution {0} found in {1}[ns]".format(S[1],tns100))

Solution 0 found in 5522799[ns]


In [143]:
print(
    "Cpu Time \n\tTns8:\t{0}\t[ns] \n\tTns10:\t{1}\t[ns] \n\tTns50:\t{2}\t[ns] \n\tTns100:\t{3}\t[ns]"
    .format(tns8,tns10,tns50,tns100))
print("Tns10/Tns8:\t{0}\t[ns]".format((tns10/tns8)))
print("Tb8/Tns8:\t{0}\t[ns]".format((tb8/tns8)))
print("Tb10/Tns10:\t{0}\t[ns]".format((tb10/tns8)))

Cpu Time 
	Tns8:	860060	[ns] 
	Tns10:	887888	[ns] 
	Tns50:	2004049	[ns] 
	Tns100:	5522799	[ns]
Tns10/Tns8:	1.0323558821477572	[ns]
Tb8/Tns8:	1.645661930562984	[ns]
Tb10/Tns10:	4.062544473641374	[ns]


By comparing the measured CPU time for Backtracking Solution and for Neightborhood Solution on datasets of 8 and 10 items we can observe that for both datasets we obtain better CPU time consumption for the second algorithm. Moreover, on slightly bigger datasets the Neighborhood Search has better CPU time consumption. On a dataset bigger by 1.25, the CPU time needed to find the solution is almost the same.

**Conclusion**
Neighborhood algorithm is fitted for searching solutions on bigger datasets, compared to backtracking. It can be seen from measurements that on bigger datasets, measured CPU time shows an performance improvement for Neighborhood algorithm. 

Still, the disadvantage of the algorithm is that in can easily report local maximums as solutions.

# Tabu Search Algorithm

## Algorithm

In [144]:
def ts_compute_neighborhood(S):
    '''
    Since the neighborhood of a solution S is made of all  the solutions 
    that that can be obtaine by using only one simple transformation,
    I chose that the simple transformation to be a bit flip
    in current solution S. This means that a value of 1 in solution array represent
    an object that is in the knapsack while a value of 0 represents a missing object.
    '''
    solutions=list()
    for i in range(0,len(S)):
        new_solution=list(S)
        if S[i] == 0:
            new_solution[i] = 1
        else:
            new_solution[i] = 0
        value = cost_function(new_solution)
        solutions.append((new_solution,value))
    #best_solution=max(solutions,key=lambda item:item[1])
    #return best_solution[0],best_solution[1]
    return solutions


In [145]:
def ts(initial_solution):
    S = initial_solution
    S_best = (copy.deepcopy(S),cost_function(S))
    TL = list()
    TL.append(S_best)
    bestCandidate = copy.deepcopy(S_best)
    iteration = 0
    while iteration < number_of_iterations:
        # will obtain a list pairs <solution, value>
        neighborhood = ts_compute_neighborhood(S_best[0])
        for candidate in neighborhood:
            if candidate not in TL and candidate[1] > bestCandidate[1]:
                bestCandidate = copy.deepcopy(candidate)
        if bestCandidate[1] > S_best[1]:
            S_best = copy.deepcopy(bestCandidate)
        TL.append(bestCandidate)
        if len(TL) > tabu_tenure:
            TL.pop(0)
        iteration += 1
    return S_best


## Plots & Measurements

In [146]:
Sts8=list()
Sts10=list()
Sts50=list()
Sts100=list()

#### Dataset 8 results

In [200]:
tabu_tenure = 4
number_of_iterations = 100
dataset = open_dataset("dataset_ds8.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts8_c1 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts8_c1 = time.process_time_ns() - tts8_c1
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts8_c1))
Sts8.append(accepted_solution)

Solution 288 found in 133929240[ns]


In [201]:
tabu_tenure = 2
number_of_iterations = 100
dataset = open_dataset("dataset_ds8.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts8_c2 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts8_c2 = time.process_time_ns() - tts8_c2
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts8_c2))
Sts8.append(accepted_solution)

Solution 288 found in 135798099[ns]


In [202]:
tabu_tenure = 6
number_of_iterations = 100
dataset = open_dataset("dataset_ds8.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts8_c3 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts8_c3 = time.process_time_ns() - tts8_c3
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts8_c3))
Sts8.append(accepted_solution)

Solution 288 found in 133619554[ns]


#### Dataset 10 results

In [204]:
tabu_tenure = 5
number_of_iterations = 250
dataset = open_dataset("dataset_ds10.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts10_c1 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts10_c1 = time.process_time_ns() - tts10_c1
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts10_c1))
Sts10.append(accepted_solution)

Solution 281 found in 427776993[ns]


In [205]:
tabu_tenure = 3
number_of_iterations = 250
dataset = open_dataset("dataset_ds10.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts10_c2 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts10_c2 = time.process_time_ns() - tts10_c2
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts10_c2))
Sts10.append(accepted_solution)

Solution 305 found in 423576617[ns]


In [206]:
tabu_tenure = 7
number_of_iterations = 250
dataset = open_dataset("dataset_ds10.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts10_c3 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts10_c3 = time.process_time_ns() - tts10_c3
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts10_c3))
Sts10.append(accepted_solution)

Solution 305 found in 429707559[ns]


#### Dataset 50 Results

In [207]:
tabu_tenure = 25
number_of_iterations = 250
dataset = open_dataset("dataset_ds50.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts50_c1 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts50_c1 = time.process_time_ns() - tts50_c1
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts50_c1))
Sts50.append(accepted_solution)

Solution 0 found in 4723086370[ns]


In [208]:
tabu_tenure = 15
number_of_iterations = 250
dataset = open_dataset("dataset_ds50.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts50_c2 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts50_c2 = time.process_time_ns() - tts50_c2
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts50_c2))
Sts50.append(accepted_solution)

Solution 0 found in 4529720952[ns]


In [209]:
tabu_tenure = 35
number_of_iterations = 250
dataset = open_dataset("dataset_ds50.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts50_c3 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts50_c3 = time.process_time_ns() - tts50_c3
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts50_c3))
Sts50.append(accepted_solution)

Solution 0 found in 4931497187[ns]


#### Dataset 100 Solutions

In [210]:
tabu_tenure = 50
number_of_iterations = 250
dataset = open_dataset("dataset_ds100.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts100_c1 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts100_c1 = time.process_time_ns() - tts100_c1
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts100_c1))
Sts100.append(accepted_solution)

Solution 0 found in 16492252209[ns]


In [211]:
tabu_tenure = 25
number_of_iterations = 250
dataset = open_dataset("dataset_ds100.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts100_c2 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts100_c2 = time.process_time_ns() - tts100_c2
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts100_c2))
Sts100.append(accepted_solution)

Solution 0 found in 15342806172[ns]


In [212]:
tabu_tenure = 75
number_of_iterations = 250
dataset = open_dataset("dataset_ds100.pickle")
initial_solutions=[generate_solution() for x in range(0,10)]
found_solutions=list()
tts100_c3 = time.process_time_ns()
for i in range(0,10):
    solution=ts(initial_solutions[i])
    found_solutions.append(solution[1])
accepted_solution = max(found_solutions)
tts100_c3 = time.process_time_ns() - tts100_c3
print("Solution {0} found in {1}[ns]".format(accepted_solution,tts100_c3))
Sts100.append(accepted_solution)

Solution 0 found in 18020002893[ns]


# Genetic Algorithm

### Algorithm


In [173]:
def generate_individual():
    return [np.random.randint(2) for x in range(0, dataset.N)]

def generate_population(population_size):
    return [generate_individual() for x in range(0,population_size)]


In [174]:
def fitness(individual):
    total_value=0
    total_weight=0
    for i in range(0,len(individual)):
        if individual[i]==1:
            total_value += dataset.v[i]
            total_weight += dataset.g[i]
    if total_weight > dataset.G:
        return 0
    else:
        return total_value


In [175]:
def crossover(first_individual,second_individual,gene_length):
    new_individual = list()
    for i in range(0,gene_length):
        crossover_value= np.random.uniform()
        if crossover_value < CROSSOVER_RATE:
            new_individual.append(first_individual[i])
        else:
            new_individual.append(second_individual[i])
    return new_individual


In [176]:
def mutate(individual,gene_length):
    for i in range(0,gene_length):
        value=np.random.uniform()
        if value < MUTATION_RATE:
            gene = round(value)
            individual[i]=gene
    return individual


In [177]:
def tournament_selection(population):
    tournament_population=list()
    for i in range (0,TOURNAMENT_SIZE):
        random_index=np.random.randint(len(population)-1)
        tournament_population.append(population[random_index])
    #get fittest individual
    tournament_population=sorted(tournament_population, key=lambda x: fitness(x),reverse=True)
    return tournament_population[0]


In [178]:
def evolve(population):
    new_population = list()
    elite_offset = 0
    if np.random.uniform() < CLONING_RATE:
        new_population.append(population[0])
        elite_offset = 1
    # Prepare for crossover
    for i in range(elite_offset,len(population)):
        first_individual = tournament_selection(population)
        second_individual = tournament_selection(population)
        new_individual= crossover(first_individual,second_individual,dataset.N)
        new_population.append(new_individual)
    #Mutate
    for i in range(elite_offset,len(population)):
        new_population[i]=mutate(new_population[i],dataset.N)
    return new_population


In [179]:
def ga():
    #generate initial population
    population = generate_population(POPULATION_SIZE)
    for i in range(0,NUMBER_OF_GENERATIONS):
        #print("Generation {0} with {1}".format(i, len(population)))
        #Sort Population
        population = sorted(population, key=lambda x: fitness(x),reverse=True)
        #Print Population
        #for ind in population:
        #    print("\t {0} , fitness {1}".format(str(ind),fitness(ind)))
        population = evolve(population)
    population = sorted(population, key=lambda x: fitness(x), reverse=True)
    return fitness(population[0])
    #print("Best generation with total value {0}".format(fitness(population[0])))


### Plots & Measurements


In [237]:
GAS8=list()
GAS10=list()
GAS50=list()
GAS100=list()

#### Configuration 1

In [238]:
NUMBER_OF_GENERATIONS = 50
POPULATION_SIZE = 1000
CLONING_RATE = 0.05
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.15
TOURNAMENT_SIZE = 5

In [239]:
dataset = open_dataset("dataset_ds8.pickle")
tga8_c1=time.process_time_ns()
S=ga()
tga8_c1=time.process_time_ns() - tga8_c1
print("Solution {0} found in {1}[ns]".format(S,tga8_c1))
GAS8.append(S)

Solution 288 found in 11421099910[ns]


In [240]:
dataset = open_dataset("dataset_ds10.pickle")
tga10_c1=time.process_time_ns()
S=ga()
tga10_c1=time.process_time_ns() - tga10_c1
print("Solution {0} found in {1}[ns]".format(S,tga10_c1))
GAS10.append(S)

Solution 305 found in 11864821076[ns]


In [241]:
dataset = open_dataset("dataset_ds50.pickle")
tga50_c1=time.process_time_ns()
S=ga()
tga50_c1=time.process_time_ns() - tga50_c1
print("Solution {0} found in {1}[ns]".format(S,tga50_c1))
GAS50.append(S)

Solution 1086 found in 21274867338[ns]


In [242]:
dataset = open_dataset("dataset_ds100.pickle")
tga100_c1=time.process_time_ns()
S=ga()
tga100_c1=time.process_time_ns() - tga100_c1
print("Solution {0} found in {1}[ns]".format(S,tga100_c1))
GAS100.append(S)

Solution 1681 found in 32371993995[ns]


#### Configuration 2

In [243]:
NUMBER_OF_GENERATIONS = 1000
POPULATION_SIZE = 50
CLONING_RATE = 0.1
CROSSOVER_RATE = 0.6
MUTATION_RATE = 0.3
TOURNAMENT_SIZE = 5

In [244]:
dataset = open_dataset("dataset_ds8.pickle")
tga8_c2=time.process_time_ns()
S=ga()
tga8_c2=time.process_time_ns() - tga8_c2
print("Solution {0} found in {1}[ns]".format(S,tga8_c2))
GAS8.append(S)

Solution 209 found in 10207837717[ns]


In [245]:
dataset = open_dataset("dataset_ds10.pickle")
tga10_c2=time.process_time_ns()
S=ga()
tga10_c2=time.process_time_ns() - tga10_c2
print("Solution {0} found in {1}[ns]".format(S,tga10_c2))
GAS10.append(S)

Solution 204 found in 10690700033[ns]


In [246]:
dataset = open_dataset("dataset_ds50.pickle")
tga50_c2=time.process_time_ns()
S=ga()
tga50_c2=time.process_time_ns() - tga50_c2
print("Solution {0} found in {1}[ns]".format(S,tga50_c2))
GAS50.append(S)

Solution 256 found in 17133198346[ns]


In [247]:
dataset = open_dataset("dataset_ds100.pickle")
tga100_c2=time.process_time_ns()
S=ga()
tga100_c2=time.process_time_ns() - tga100_c2
print("Solution {0} found in {1}[ns]".format(S,tga100_c2))
GAS100.append(S)

Solution 257 found in 25197287432[ns]


#### Configuration 3

In [248]:
NUMBER_OF_GENERATIONS = 150
POPULATION_SIZE = 1000
CLONING_RATE = 0.02
CROSSOVER_RATE = 0.75
MUTATION_RATE = 0.23
TOURNAMENT_SIZE = 5

In [249]:
dataset = open_dataset("dataset_ds8.pickle")
tga8_c3=time.process_time_ns()
S=ga()
tga8_c3=time.process_time_ns() - tga8_c3
print("Solution {0} found in {1}[ns]".format(S,tga8_c3))
GAS8.append(S)

Solution 288 found in 33780081479[ns]


In [250]:
dataset = open_dataset("dataset_ds10.pickle")
tga10_c3=time.process_time_ns()
S=ga()
tga10_c3=time.process_time_ns() - tga10_c3
print("Solution {0} found in {1}[ns]".format(S,tga10_c3))
GAS10.append(S)

Solution 305 found in 35080996880[ns]


In [251]:
dataset = open_dataset("dataset_ds50.pickle")
tga50_c3=time.process_time_ns()
S=ga()
tga50_c3=time.process_time_ns() - tga50_c3
print("Solution {0} found in {1}[ns]".format(S,tga50_c3))
GAS50.append(S)

Solution 870 found in 59016475810[ns]


In [252]:
dataset = open_dataset("dataset_ds100.pickle")
tga100_c3=time.process_time_ns()
S=ga()
tga100_c3=time.process_time_ns() - tga100_c3
print("Solution {0} found in {1}[ns]".format(S,tga100_c3))
GAS100.append(S)

Solution 1290 found in 86849431654[ns]


# General Opinions

10. Compare S[] GA-i with S[] TS-i and both of them with SNS-i and SB-i (for i = 8, 10, 50 and 100).


From measurement it resulted that GA algorithm obtained the best solutions for the problem. Compared to TS, GA's results were more closer to the problem's solution. TS seemed to be stuck, at least on bigger datasets in local maximums or it failed to find the correct solution. 

In term of results, a comparison between GA or TS and NS or B, the measurement shows that the results obtained after running GA and TS were better on bigger datasets although on small datasets they didn't perform well from the point of view of CPU time.


# GA vs TS vs B on small Datasets

In [213]:
print("Tga10_c1/Tga8_c1:\t {0}/{1}={2}".format(tga10_c1,tga8_c1,(tga10_c1/tga8_c1)))
print("Tga10_c2/Tga8_c2:\t {0}/{1}={2}".format(tga10_c2,tga8_c2,(tga10_c2/tga8_c2)))
print("Tga10_c3/Tga8_c3:\t {0}/{1}={2}".format(tga10_c3,tga8_c3,(tga10_c3/tga8_c3)))

Tga10_c1/Tga8_c1:	 11735988628/11405317045=1.0289927567725936
Tga10_c2/Tga8_c2:	 10798685449/10452285206=1.0331411013164042
Tga10_c3/Tga8_c3:	 35318772359/34019192508=1.0382013726720407


By comparing the CPU time values obtained after measuring all GA with 3 possible configurations performing on dataset 8 and dataset 10 we can observe that the performance is almost the same.

Considering that in the case of backtracking algorithm, for a datasize bigger by 1.25 times the degradation of performance was more than double.

In [214]:
print("Tts10_c1/Tts8_c1:\t {0}/{1}={2}".format(tts10_c1,tts8_c1,(tts10_c1/tts8_c1)))
print("Tts10_c2/Tts8_c2:\t {0}/{1}={2}".format(tts10_c2,tts8_c2,(tts10_c2/tts8_c2)))
print("Tts10_c3/Tts8_c3:\t {0}/{1}={2}".format(tts10_c3,tts8_c3,(tts10_c3/tts8_c3)))

Tts10_c1/Tts8_c1:	 427776993/133929240=3.1940522696910696
Tts10_c2/Tts8_c2:	 423576617/135798099=3.119164554726204
Tts10_c3/Tts8_c3:	 429707559/133619554=3.2159032576923585


From the obtained measured values it can be observed that by increasing the dataset size
the performance dropped 3 times. Furthermore, it can be observed that on small datasets, the performance degradation of TS algorithm is comparable with the one obtained for Backtracking.

In [225]:
print("Tb8/Tga8_c1:\t {0}/{1}={2}".format(tb8,tga8_c1,(tb8/tga8_c1)))
print("Tb8/Tga8_c2:\t {0}/{1}={2}".format(tb8,tga8_c2,(tb8/tga8_c2)))
print("Tb8/Tga8_c3:\t {0}/{1}={2}".format(tb8,tga8_c3,(tb8/tga8_c3)))

Tb8/Tga8_c1:	 1013538/11405317045=8.886539462261831e-05
Tb8/Tga8_c2:	 1013538/10452285206=9.696807731750293e-05
Tb8/Tga8_c3:	 1013538/34019192508=2.979312338944127e-05


In [226]:
print("Tb10/Tga10_c1:\t {0}/{1}={2}".format(tb10,tga10_c1,(tb10/tga10_c1)))
print("Tb10/Tga10_c2:\t {0}/{1}={2}".format(tb10,tga10_c2,(tb10/tga10_c2)))
print("Tb10/Tga10_c3:\t {0}/{1}={2}".format(tb10,tga10_c3,(tb10/tga10_c3)))

Tb10/Tga10_c1:	 3516280/11735988628=0.0002996151505814155
Tb10/Tga10_c2:	 3516280/10798685449=0.00032562111532988687
Tb10/Tga10_c3:	 3516280/35318772359=9.955838680513974e-05


Although GA is a great algorithm that can be applied on bigger datasets, it can be observed from measurements that, compared with backtracking, on small datasets is not feasible.

In [227]:
print("Tb8/Tts8_c1:\t {0}/{1}={2}".format(tb8,tts8_c1,(tb8/tts8_c1)))
print("Tb8/Tts8_c2:\t {0}/{1}={2}".format(tb8,tts8_c2,(tb8/tts8_c2)))
print("Tb8/Tts8_c3:\t {0}/{1}={2}".format(tb8,tts8_c3,(tb8/tts8_c3)))

Tb8/Tts8_c1:	 1013538/133929240=0.007567712621978591
Tb8/Tts8_c2:	 1013538/135798099=0.00746356545094199
Tb8/Tts8_c3:	 1013538/133619554=0.007585252080694716


In [228]:
print("Tb10/Tts10_c1:\t {0}/{1}={2}".format(tb10,tts10_c1,(tb10/tts10_c1)))
print("Tb10/Tts10_c2:\t {0}/{1}={2}".format(tb10,tts10_c2,(tb10/tts10_c2)))
print("Tb10/Tts10_c3:\t {0}/{1}={2}".format(tb10,tts10_c3,(tb10/tts10_c3)))

Tb10/Tts10_c1:	 3516280/427776993=0.008219890404437904
Tb10/Tts10_c2:	 3516280/423576617=0.008301402529970156
Tb10/Tts10_c3:	 3516280/429707559=0.008182960542241706


It can be seen that on small datasets, compared with backtracking, TS is not a feasible algorithm.

# GA vs NS vs TS on bigger datasets

In [230]:
print("Tga50_c1/tns50:\t {0}/{1}={2}".format(tga50_c1,tns50,(tga50_c1/tns50)))
print("Tga50_c2/tns50:\t {0}/{1}={2}".format(tga50_c2,tns50,(tga50_c2/tns50)))
print("Tga50_c3/tns50:\t {0}/{1}={2}".format(tga50_c3,tns50,(tga50_c3/tns50)))

Tga50_c1/tns50:	 21242617744/2004049=10599.84947673435
Tga50_c2/tns50:	 17812072853/2004049=8888.042584288109
Tga50_c3/tns50:	 58750305317/2004049=29315.802815699615


In [231]:
print("Tga100_c1/tns100:\t {0}/{1}={2}".format(tga100_c1,tns100,(tga100_c1/tns100)))
print("Tga100_c2/tns100:\t {0}/{1}={2}".format(tga100_c2,tns100,(tga100_c2/tns100)))
print("Tga100_c3/tns100:\t {0}/{1}={2}".format(tga100_c3,tns100,(tga100_c3/tns100)))

Tga100_c1/tns100:	 32917133007/5522799=5960.226509601382
Tga100_c2/tns100:	 26302340789/5522799=4762.501910534857
Tga100_c3/tns100:	 84904569232/5522799=15373.467191545446


Compared with Neighborhood search, Genetic algorithm performed worse, in terms of CPU time, on big datasets but the results obtained were optimal or almost optimal.

In [232]:
print("Tts50_c1/tns50:\t {0}/{1}={2}".format(tts50_c1,tns50,(tts50_c1/tns50)))
print("Tts50_c2/tns50:\t {0}/{1}={2}".format(tts50_c2,tns50,(tts50_c2/tns50)))
print("Tts50_c3/tns50:\t {0}/{1}={2}".format(tts50_c3,tns50,(tts50_c3/tns50)))

Tts50_c1/tns50:	 4723086370/2004049=2356.771900287867
Tts50_c2/tns50:	 4529720952/2004049=2260.2845299690775
Tts50_c3/tns50:	 4931497187/2004049=2460.7667711717627


In [233]:
print("Tts100_c1/tns100:\t {0}/{1}={2}".format(tts100_c1,tns100,(tts100_c1/tns100)))
print("Tts100_c2/tns100:\t {0}/{1}={2}".format(tts100_c2,tns100,(tts100_c2/tns100)))
print("Tts100_c3/tns100:\t {0}/{1}={2}".format(tts100_c3,tns100,(tts100_c3/tns100)))

Tts100_c1/tns100:	 16492252209/5522799=2986.212644892563
Tts100_c2/tns100:	 15342806172/5522799=2778.08520136257
Tts100_c3/tns100:	 18020002893/5522799=3262.838805649092


Like GA, compared with NS, TS performed worse, in terms of CPU time, on big datasets but the results obtained were optimal or almost optimal.

In [234]:
print("Tga50_c1/tts50_c1:\t {0}/{1}={2}".format(tga50_c1,tts50_c1,(tga50_c1/tts50_c1)))
print("Tga50_c2/tts50_c2:\t {0}/{1}={2}".format(tga50_c2,tts50_c2,(tga50_c2/tts50_c2)))
print("Tga50_c3/tts50_c3:\t {0}/{1}={2}".format(tga50_c3,tts50_c3,(tga50_c3/tts50_c3)))

Tga50_c1/tts50_c1:	 21242617744/4723086370=4.497613653421291
Tga50_c2/tts50_c2:	 17812072853/4529720952=3.932267140017856
Tga50_c3/tts50_c3:	 58750305317/4931497187=11.913279697669227


In [235]:
print("Tga100_c1/tts100_c1:\t {0}/{1}={2}".format(tga100_c1,tts100_c1,(tga100_c1/tts100_c1)))
print("Tga100_c2/tts100_c2:\t {0}/{1}={2}".format(tga100_c2,tts100_c2,(tga100_c2/tts100_c2)))
print("Tga100_c3/tts100_c3:\t {0}/{1}={2}".format(tga100_c3,tts100_c3,(tga100_c3/tts100_c3)))

Tga100_c1/tts100_c1:	 32917133007/16492252209=1.995914966000627
Tga100_c2/tts100_c2:	 26302340789/15342806172=1.7143109607289901
Tga100_c3/tts100_c3:	 84904569232/18020002893=4.7116845505602996


On big datasets, depending on the configurations, GA performed almost identical with TS or worse. Though, the advantage of GA is the optimality of the solutions obtained.